# K-nearest neighbor

First off, let's do the preprocessing presented in [Data Process](./dataProcess.ipynb):

In [1]:
%%capture

import sys
!{sys.executable} -m pip install nbformat

%run -G ./dataProcess.ipynb

In [6]:
# imports
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import log_loss
from sklearn.metrics import recall_score
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA

### PCA
From [firstModel](./firstModel.ipynb) we found that PCA with n_components=7 was a reasonable step to take, reducing the model's complexity and reducing the risk of overfitting.

In [10]:
# oversample
smote = SMOTE(random_state=42)
df_overSampled, y_overSampled = smote.fit_resample(X=df, y=df['Class'])
df_overSampled = df_overSampled.drop(['Class'], axis=1)

# PCA
pca = PCA(n_components=7)
pca.fit(df_overSampled)
X = pca.transform(df_overSampled)
